Студент Жилкин Федор, 344
# Домашняя работа 4: "Численное решение интегрального уравнения Фредгольма 2-го рода."
**Задания:**

1. Метод механических квадратур  
2. Метод замены ядра на вырожденное

$u(x) + 0.5\int\limits_0^1 e^{xy}u(y)dy = x + 0.5$

In [62]:
import numpy as np
import pandas as pd 
import sympy

In [251]:
# Функция f(x) справа
def f(x):
    return x + 0.5 

In [252]:
# Ядро
def K(x, y):
    return np.exp(x*y)

## Метод механических квадратур

In [329]:
# Метод механических квадратур
def mechanical_quadrature(n, X):
     
    """
    Решение интегрального уравнения методом механических квадратур
    n -- кол-во точек разбиения отрезка
    X -- точка
    default: up, down = 1, 0
    """
    
    # Задаем равномерную сетку
    up, down = 1, 0
    h = (up - down)/n
    x = [(down + i*h) for i in range(0, n)]
    
    A = np.zeros((n, n))
    B = np.zeros(n)
    
    # Веса составной формулы средних прямоугольников
    weights = np.ones(n)*h 
    weights[0] /= 2
    weights[n-1] /= 2
    
    for i in range(0, n):
        for j in range(0, n):
            if j == i:
                beta = 1
            else:
                beta = 0
            A[i][j] = beta - weights[j]*K(x[i], x[j])
        B[i] = f(x[i])
    
    # Нашли искомый вектор Z
    Z = np.linalg.solve(A, B)
    
    # Решение уравнения
    sum_ = sum([weights[i]*K(X, x[i])*Z[i] for i in range(0, n)])
    u = -0.5*sum_ + f(X)
    return u

In [363]:
mechanical_quadrature(4800, 0.1)

2.1368947678067136

In [417]:
degenerate_kernel(60, 0.1)

2.135295269050406

Ядро $e^{xy}$ заменяем на вырожденное ядро: $e^{xy} = \sum\limits^{n}_{k=0}{\frac{(xy)^{k}}{k!}} = \sum\limits^{n}_{k=0}x^k \frac{y^k}{k!} = \sum\limits^{n}_{k=0}a_k(x)b_k(y)$

Отсюда $a_k(x) = x^k$

In [402]:
def integrate(i, j):
    """
    Вычисляем интеграл ∫[y^(i+j)/i!]dy
    default: up, down = 1, 0
    """
    up, down = 1, 0
    y = sympy.Symbol('y')
    d = sympy.integrate(y**(i+j)/math.factorial(i), y)
    res = sympy.lambdify(y, d, 'numpy')
    return res(up) - res(down)

In [408]:
def degenerate_kernel(n, X):
    
    """
    Решение интегрального уравнения методом замены ядра на вырожденное
    n -- кол-во точек разбиения отрезка
    X -- точка
    default: up, down = 1, 0
    """
    
    # Задаем равномерную сетку
    up, down = 1, 0
    h = (up - down)/n
    x = [(down + i*h) for i in range(0, n)]
    
    A = np.zeros((n, n))
    B = np.zeros(n)
    
    for i in range(0, n):
        for j in range(0, n):
            if j == i:
                beta = 1
            else:
                beta = 0
            A[i][j] = beta - integrate(i, j)
        B[i] = integrate_2(i)
        
    # Обратная матрица к А
    D = np.linalg.inv(A)
    
    # Искомый вектор C
    #C = np.zeros(n)
    #for i in range(0, n):
     #   C[i] = sum([D[i][j]*B[j] for j in range(0, n)])
    C = np.linalg.solve(A, B)
    sum_ = sum([C[i]*(X**i) for i in range(0,n)])
    u = f(X) - 0.5*sum_
    return u

In [405]:
def integrate_2(i):
    """
    Вычисляем интеграл ∫[y^(i+j)/i!]dy
    default: up, down = 1, 0
    """
    up, down = 1, 0
    y = sympy.Symbol('y')
    d = sympy.integrate(y**(i)/math.factorial(i) * (y + 0.5), y)
    res = sympy.lambdify(y, d, 'numpy')
    return res(up) - res(down)

In [406]:
integrate_2(2)

0.20833333333333331

In [407]:
f(0.3)

0.8